<a href="https://colab.research.google.com/github/bogard75/ali_items/blob/main/alipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-translate')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
B

In [70]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup

driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [71]:
from collections import namedtuple

def item_counts(html, search):
    driver.get(html+search)
    driver.implicitly_wait(1) # seconds
    
    res = driver.find_element_by_class_name('next-breadcrumb-text.activated').text
    print('results found : {}'.format(res))
    
    try:
        res = int(res.split('(')[-1].split()[0].replace(',',''))
    except:
        res = 0
    return res

def search_items(html, search):
    # read results count
    results = item_counts(html, search)
    items_per_page = 60
    total_pages = int(results / items_per_page) + 1
    
    res = {}
    for page in range(1,total_pages+1):
        print('page {}---------------------------'.format(page))
        s_ = search + '{0}'.format(page)
        driver.get(html+s_)
        driver.implicitly_wait(1) # seconds
        
        # select body
        #body = driver.find_element_by_css_selector('body')
        #body.click()
        # pass new user coupon
        try:
            new_user = driver.find_element_by_xpath("/html/body/div[7]/div[2]/div/a") # "//next-dialog-close|//newuser-container"
        except NoSuchElementException:
            pass 
        else:
            print("new user clicked ----------")
            new_user.click()
        
        # scroll to bottom
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        
        for i in range(0,10):
            body = driver.find_element_by_tag_name('body')
            body.send_keys(Keys.PAGE_DOWN)
            driver.implicitly_wait(1) # seconds
        
        #    sleep(1)
        items = driver.find_elements_by_class_name("list-item")
        
        for (n, i) in enumerate(items):
            print('{}-----------'.format(n))
            print(i.text)
            print(i.find_element_by_tag_name('a').get_attribute('href'))
            res['p{:02d}_i{:02d}'.format(page, n)] = i.find_element_by_tag_name('a').get_attribute('href')
            
    return res

def item_info(html):
    driver.get(html)
    driver.implicitly_wait(1) # seconds
    
    Product = namedtuple('product', 'item address title price reviewer shipping')

    address_ = html
    item_ = html.split('/')[4].split('?')[0]
    try:
        title_ = driver.find_element_by_class_name('product-title').text
    except NoSuchElementException:
        title_ = ''
    try:
        reviewer_ = driver.find_element_by_class_name('product-reviewer').text
    except NoSuchElementException:
        reviewer_ = ''
    try:
        price_ = driver.find_element_by_class_name('product-price').text
    except NoSuchElementException:
        price_ = ''
    try:
        shipping_ = driver.find_element_by_class_name('product-shipping-price').text
    except NoSuchElementException:
        shipping_ = ''
    res = Product(address=address_, item=item_, title=title_, reviewer=reviewer_, price=price_, shipping=shipping_)
    
    print(res)
    return res



In [ ]:
# mens clothing 
#html = 'https://www.aliexpress.com/category/100003070/men-clothing.html' 

#cellphones-telecommunications
# https://www.aliexpress.com는 redirect이전 페이지에서 계속 읽음?
# groupsort=1 newest로 하면 count가 리프레쉬
html = 'https://aliexpress.com/category/205000143/cellphones-telecommunications.html?groupsort=1'
search = 'minPrice=5000&maxPrice=&page='
#results = item_counts(html, search) # 563
res1 = search_items(html, search)

search = 'minPrice=4000&maxPrice=5000&page='  # 400
#results = item_counts(html, search)
res2 = search_items(html, search)

search = 'minPrice=3000&maxPrice=4000&page='  # 766
#results = item_counts(html, search)
res3 = search_items(html, search)

search = 'minPrice=2500&maxPrice=3000&page='  # 689
#results = item_counts(html, search)
res4 = search_items(html, search)

search = 'minPrice=2000&maxPrice=2500&page='  # 1242
#results = item_counts(html, search)
res5 = search_items(html, search)

search = 'minPrice=1700&maxPrice=2000&page='  # 1121
#results = item_counts(html, search)
res6 = search_items(html, search)

search = 'minPrice1500&maxPrice=1700&page='  # 1147
#results = item_counts(html, search)
res7 = search_items(html, search)


# item address list
res = [res1, res2, res3, res4, res5, res6, res7]
flatten = lambda t : [item for sublist in t for item in sublist]
addrs = list(map(lambda x : [ref for i, ref in x.items()], res))
addrs = flatten(addrs)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Cubot Smartphone 64GB LTE/WCDMA/GSM Octa Core Fingerprint Recognition/face Recognition
US $89.99
US $179.9950% off
Free Shipping
4.7
1136 Sold
Cubot Official Store
https://www.aliexpress.com/item/4000179658201.html?algo_pvid=6dcb389b-d31d-488e-aca1-73c6098bd645&algo_expid=6dcb389b-d31d-488e-aca1-73c6098bd645-5&btsid=0bb0624316091964239587674e545a&ws_ab_test=searchweb0_0,searchweb201602_,searchweb201603_
6-----------
J4859C - 1000Base-LX SFP 1310nm 10km (Compatible with HP)
US $11
Free Shipping
4.3
6 Sold
Optical Transceiver Store
https://www.aliexpress.com/item/32225339507.html?algo_pvid=6dcb389b-d31d-488e-aca1-73c6098bd645&algo_expid=6dcb389b-d31d-488e-aca1-73c6098bd645-6&btsid=0bb0624316091964239587674e545a&ws_ab_test=searchweb0_0,searchweb201602_,searchweb201603_
7-----------
Protection-Box Drop-Cable Splice-Tray Optical-Fiber 1 To 50pcs 2-Out Heat-Shrink-Tubing
US $15.30
+ Shipping: US $4.68
5.0
1 Sold
AUA Store
https://www.aliexpress.com/item/40

In [ ]:

item_infos = [item_info(addr) for addr in addrs]
   

product(item='1005001940664435.html', address='https://www.aliexpress.com/item/1005001940664435.html?algo_pvid=a06891c7-447c-4552-903c-9f50c02d734b&algo_expid=a06891c7-447c-4552-903c-9f50c02d734b-0&btsid=0b0a555a16091680501558935e13ed&ws_ab_test=searchweb0_0,searchweb201602_,searchweb201603_', title='2020 New Women Hoodies Round Neck Long Sleeve Pullover Female Hoodie Gothic Women Sweatshirts Streetwear', price='US $2,599.00', reviewer='', shipping='Shipping: US $10.59 ')
product(item='4001227097611.html', address='https://www.aliexpress.com/item/4001227097611.html?algo_pvid=a06891c7-447c-4552-903c-9f50c02d734b&algo_expid=a06891c7-447c-4552-903c-9f50c02d734b-1&btsid=0b0a555a16091680501558935e13ed&ws_ab_test=searchweb0_0,searchweb201602_,searchweb201603_', title='summer new 2020 - Tagless Youth T-Shirt - 5450 Cotton O-Neck China (Mainland)', price='US $1,688.00 / lot (5 pieces)', reviewer='5.0\n1 Review\n1 order', shipping='Shipping: US $4.00 ')
product(item='32951648741.html', address=